In [17]:
import os
import streamlit as st
from langchain_groq import ChatGroq
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from dotenv import load_dotenv

In [18]:
os.environ["GROQ_API"] = os.getenv("GROQ_API")

groq_api=os.getenv("GROQ_API")

llm=ChatGroq(groq_api_key=groq_api,model_name='Llama3-8b-8192')
prompt=ChatPromptTemplate.from_template(
    """
    Answer the question based on the context provided.
    Please provide the most accurate answer response based on the question and context.
    <context>
    {context}
    </context>
    Question:{input}

    """ 

)

In [ ]:
embeddings = OllamaEmbeddings()
loader = PyPDFDirectoryLoader("pdfs")
docs = st.session_state.loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents=st.session_state.text_splitter.split_documents(st.session_state.docs[:50])
vectors=FAISS.from_documents(st.session_state.final_documents,st.session_state.embeddings)

In [20]:
vectorstore = FAISS.load_local(
    "vectors", 
    embeddings, 
    allow_dangerous_deserialization=True
)


In [21]:
user_prompt="what is noisy data"
document_chain=create_stuff_documents_chain(llm,prompt)
retriever=vectorstore.as_retriever()
retrievel_chain=create_retrieval_chain(retriever,document_chain)
response=retrievel_chain.invoke({'input':user_prompt})
response['answer']

'Based on the context provided, noisy data is mentioned in the Data Cleaning as a Process section (Data scrubbing: use simple domain knowledge (e.g., postal code, spell-check) to detect errors and make corrections). Noisy data likely refers to data that contains errors or inconsistencies, such as typos, incorrect values, or irrelevant information. Data scrubbing is a process used to detect and correct these errors to improve data quality.'